In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import MinMaxScaler , FunctionTransformer, PowerTransformer, KBinsDiscretizer
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

2026-01-06 02:07:49.750316: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-06 02:07:49.788378: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-06 02:07:50.746010: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/devil/DL/myenv/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: Future

In [ ]:
CSV_FILE = "/home/devil/DL/CDC_Project/train(1)(train(1)).csv"
IMAGE_DIR = Path("/home/devil/DL/ultra_high_res_images")
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50
PREFETCH_SIZE = 2  

FEATURE_COLUMNS = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront','lat','long','view','condition','grade','sqft_basement','basement_present','age','is_renovated','living_lot_ratio','years_since_renovation','sqft_living15','sqft_lot15']

In [ ]:
df = pd.read_csv(CSV_FILE)
df['age'] = 2015 - df['yr_built']
df['is_renovated'] = (df['yr_renovated'] > 0).astype(int)
df['basement_present'] = (df['sqft_basement'] > 0).astype(int)
df['living_lot_ratio'] = df['sqft_living'] / df['sqft_lot']
df['years_since_renovation'] = np.where(
    df['yr_renovated'] > 0,
    2015 - df['yr_renovated'],
    0
)


data_index = []
for idx, row in df.iterrows():
    property_id = row['id']
    
    img_path = None
    for ext in ['.jpg', '.png', '_hires.tif', '.tif']:
        path = IMAGE_DIR / f"property_{property_id}{ext}"
        if path.exists():
            img_path = str(path)
            break
    
    if img_path:
        data_index.append({
            'img_path': img_path,
            'features': row[FEATURE_COLUMNS].values.astype(np.float32),
            'price': float(row['price'])
        })
    
    if (idx + 1) % 1000 == 0:
        print(f"  Indexed {idx + 1}/{len(df)}...")

print(f"Found {len(data_index)} properties with images")

if len(data_index) == 0:
    print("No images found!")
    exit(1)

  Indexed 1000/16209...
  Indexed 2000/16209...
  Indexed 3000/16209...
  Indexed 4000/16209...
  Indexed 5000/16209...
  Indexed 6000/16209...
  Indexed 7000/16209...
  Indexed 8000/16209...
  Indexed 9000/16209...
  Indexed 10000/16209...
  Indexed 11000/16209...
  Indexed 12000/16209...
  Indexed 13000/16209...
  Indexed 14000/16209...
  Indexed 15000/16209...
  Indexed 16000/16209...
Found 16209 properties with images


In [4]:
train_idx, temp_idx = train_test_split(
    range(len(data_index)), test_size=0.2, random_state=42
)
val_idx, test_idx = train_test_split(
    temp_idx, test_size=0.5, random_state=42
)

print(f"  Train: {len(train_idx)}")
print(f"  Val: {len(val_idx)}")
print(f"  Test: {len(test_idx)}")

# Extract data for each split
train_data = [data_index[i] for i in train_idx]
val_data = [data_index[i] for i in val_idx]
test_data = [data_index[i] for i in test_idx]

  Train: 12967
  Val: 1621
  Test: 1621


In [5]:
train_features = np.array([item['features'] for item in train_data])

scaler = StandardScaler()
scaler.fit(train_features)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Normalize all features
for item in train_data:
    item['features'] = scaler.transform(item['features'].reshape(1, -1))[0]
for item in val_data:
    item['features'] = scaler.transform(item['features'].reshape(1, -1))[0]
for item in test_data:
    item['features'] = scaler.transform(item['features'].reshape(1, -1))[0]

In [ ]:
def load_image(img_path):
    img_path_str = img_path.decode('utf-8')
    
    try:
        if img_path_str.endswith('.tif'):
            import rasterio
            with rasterio.open(img_path_str) as src:
                if src.count >= 3:
                    r = src.read(1)
                    g = src.read(2)
                    b = src.read(3)
                    img = np.dstack([r, g, b])
                else:
                    band = src.read(1)
                    img = np.stack([band]*3, axis=-1)
                
                img = img.astype(float)
                p2, p98 = np.percentile(img, (2, 98))
                if p98 > p2:
                    img = np.clip((img - p2) / (p98 - p2), 0, 1)
                    img = (img * 255).astype(np.uint8)
                else:
                    img = np.zeros((*IMAGE_SIZE, 3), dtype=np.uint8)
        else:
            img = cv2.imread(img_path_str)
            if img is None:
                return np.zeros((*IMAGE_SIZE, 3), dtype=np.float32)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        img = cv2.resize(img, IMAGE_SIZE)
        return img.astype(np.float32) / 255.0
    except:
        return np.zeros((*IMAGE_SIZE, 3), dtype=np.float32)


def data_generator(data_list):
    for item in data_list:
        img_path = item['img_path']
        features = item['features']
        price = np.log1p(item['price'])


        img = load_image(img_path.encode('utf-8'))
        
        yield img, features, price


def create_dataset(data_list, shuffle=True):

    dataset = tf.data.Dataset.from_generator(
        lambda: data_generator(data_list),
        output_signature=(
            tf.TensorSpec(shape=(*IMAGE_SIZE, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(len(FEATURE_COLUMNS),), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.float32)
        )
    )

    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000)

    dataset = dataset.batch(BATCH_SIZE)

    dataset = dataset.map(
        lambda img, feat, price: ((img, feat), price),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    
    return dataset

print("  Creating training dataset...")
train_dataset = create_dataset(train_data, shuffle=True)

print("  Creating validation dataset...")
val_dataset = create_dataset(val_data, shuffle=False)

print("  Creating test dataset...")
test_dataset = create_dataset(test_data, shuffle=False)

print("✓ Datasets created")

train_steps = len(train_data) // BATCH_SIZE
val_steps = len(val_data) // BATCH_SIZE
test_steps = len(test_data) // BATCH_SIZE

print(f"  Train steps per epoch: {train_steps}")
print(f"  Val steps per epoch: {val_steps}")

  Creating training dataset...
  Creating validation dataset...
  Creating test dataset...
✓ Datasets created
  Train steps per epoch: 405
  Val steps per epoch: 50


I0000 00:00:1767645489.921654    2452 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4302 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 6GB Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [ ]:
def create_model():
    img_input = layers.Input(shape=(224, 224, 3), name='image')
    
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(img_input)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    img_branch = layers.Dropout(0.4)(x)

    feat_input = layers.Input(shape=(len(FEATURE_COLUMNS),), name='features')
    f = layers.Dense(64, activation='relu')(feat_input)
    f = layers.Dropout(0.3)(f)
    feat_branch = layers.Dense(32, activation='relu')(f)
    
    combined = layers.concatenate([img_branch, feat_branch])
    z = layers.Dense(64, activation='relu')(combined)
    z = layers.Dropout(0.3)(z)
    output = layers.Dense(1, name='price')(z)
    
    model = models.Model(inputs=[img_input, feat_input], outputs=output)
    return model


model = create_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='mse',
    metrics=['mae']
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image (InputLayer)  │ (None, 224, 224,  │          0 │ -                 │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 224, 224,  │        896 │ image[0][0]       │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 112, 112,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 112, 112,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 56, 56,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 56, 56,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 28, 28,    │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 28, 28,    │    295,168 │ max_pooling2d_2[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ features            │ (None, 19)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ conv2d_3[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      1,280 │ features[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     32,896 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      2,080 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 160)       │          0 │ dropout[0][0],    │
│ (Concatenate)       │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 64)        │     10,304 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64)        │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ price (Dense)       │ (None, 1)         │         65 │ dropout_2[0][0]   │
└─────────────────────┴───────────────────┴────────────┴─────────────────

 Total params: 435,041 (1.66 MB)

 Trainable params: 435,041 (1.66 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        'best_model.h5',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=7,
        min_lr=1e-7,
        verbose=1
    ),
    tf.keras.callbacks.CSVLogger('training_log.csv')
]

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/50


2026-01-06 02:08:35.439123: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:453] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 468 of 1000
2026-01-06 02:08:46.666192: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:483] Shuffle buffer filled.
2026-01-06 02:08:47.264480: I external/local_xla/xla/service/service.cc:163] XLA service 0x7f96280035d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2026-01-06 02:08:47.264511: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 6GB Laptop GPU, Compute Capability 8.6
2026-01-06 02:08:47.312006: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2026-01-06 02:08:47.610927: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91701
2026-01-06 02:08:47.778272: I external/local_xla/xla/servi

      2/Unknown 34s 75ms/step - loss: 163.7766 - mae: 12.7903

I0000 00:00:1767645537.773046    3467 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


    406/Unknown 258s 553ms/step - loss: 18.2756 - mae: 3.0518

...

2026-01-06 05:47:58.571450: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 15376248340720911831
2026-01-06 05:47:58.571472: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 2160091407360948014
2026-01-06 05:48:08.590265: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:453] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 555 of 1000
2026-01-06 05:48:16.620631: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:483] Shuffle buffer filled.


405/406 ━━━━━━━━━━━━━━━━━━━━ 0s 545ms/step - loss: 0.6419 - mae: 0.6350

2026-01-06 05:51:57.395315: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 15376248340720911831
2026-01-06 05:51:57.395336: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 14588052486359974992
2026-01-06 05:51:57.395344: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 2160091407360948014



Epoch 50: val_loss did not improve from 0.04027

Epoch 50: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
406/406 ━━━━━━━━━━━━━━━━━━━━ 268s 614ms/step - loss: 0.6322 - mae: 0.6305 - val_loss: 0.0445 - val_mae: 0.1572 - learning_rate: 2.5000e-04
Restoring model weights from the end of the best epoch: 43.


2026-01-06 05:52:26.150407: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 15376248340720911831
2026-01-06 05:52:26.150426: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 14588052486359974992
2026-01-06 05:52:26.150432: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 2160091407360948014


In [ ]:
y_pred_list = []
y_true_list = []

for (img_batch, feat_batch), y_batch in test_dataset:
    preds = model.predict([img_batch, feat_batch], verbose=0)
    y_pred_list.extend(preds.flatten())
    y_true_list.extend(y_batch.numpy())

y_pred_log = np.array(y_pred_list)
y_true_log = np.array(y_true_list)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_true_log)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)
mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1))) * 100

print(f"\nTest Results:")
print(f"  MAE:  ${mae:,.2f}")
print(f"  RMSE: ${rmse:,.2f}")
print(f"  R²:   {r2:.4f}")
print(f"  MAPE: {mape:.2f}%")

print("\nSample Predictions:")
print(f"{'Actual':>15} {'Predicted':>15} {'Error':>15}")
print("-" * 47)
for i in range(min(10, len(y_true))):
    error = abs(y_true[i] - y_pred[i])
    print(f"${y_true[i]:>14,.0f} ${y_pred[i]:>14,.0f} ${error:>14,.0f}")


Test Results:
  MAE:  $81,960.50
  RMSE: $140,734.89
  R²:   0.8315
  MAPE: 15.32%

Sample Predictions:
         Actual       Predicted           Error
-----------------------------------------------
$       178,500 $       208,615 $        30,115
$     1,518,631 $     1,374,973 $       143,658
$       225,000 $       309,103 $        84,103
$       540,000 $       573,889 $        33,889
$       207,000 $       266,822 $        59,822
$       325,000 $       382,907 $        57,907
$       279,000 $       279,689 $           689
$       279,000 $       284,332 $         5,332
$       339,000 $       377,780 $        38,780
$       375,000 $       342,186 $        32,814
